**Imports**

In [3]:
import pandas as pd
import numpy as np

In [4]:
# set display widht and length

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 110)

#### CBP Benthic Data

In [5]:
cbp_benthic = pd.read_csv('/Users/bibor/Hack_the_Bay/Data/CBP_Benthic.csv')

In [6]:
# drop columns where all values are missing

for column in cbp_benthic.columns:
    if cbp_benthic[column].isna().sum() == cbp_benthic.shape[0]:
        cbp_benthic.drop(column, axis=1, inplace=True)

In [8]:
# drop Unnamed:0.1 and Unnamed: 0.1.1 columns

cbp_benthic.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1, inplace=True)

In [9]:
cbp_benthic.shape

(129671, 32)

In [10]:
cbp_benthic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129671 entries, 0 to 129670
Data columns (total 32 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Agency                       129671 non-null  object 
 1   Benthic Classification Name  129671 non-null  object 
 2   BioMethod                    129671 non-null  object 
 3   Count                        129671 non-null  object 
 4   CountyCity                   129671 non-null  object 
 5   Database                     129671 non-null  object 
 6   Date                         129671 non-null  object 
 7   EcoRegionLevel3              129671 non-null  float64
 8   EcoRegionLevel3Description   129671 non-null  object 
 9   EventId                      129671 non-null  float64
 10  FIPS                         129671 non-null  float64
 11  GMethod                      129671 non-null  float64
 12  HUC12                        129671 non-null  float64
 13 

**Variables of interest:** Benthic Classification Name (organism), EventId, ReportingValue

In [11]:
# create for loop to find ecoli in 'Benthic Classification Name'

mask = cbp_benthic['Benthic Classification Name'].str.lower().str.contains('ecoli')
cbp_benthic.loc[mask]

# no ecoli in this column

,Agency,Benthic Classification Name,BioMethod,Count,CountyCity,Database,Date,EcoRegionLevel3,EcoRegionLevel3Description,EventId,FIPS,GMethod,HUC12,HUC12Description,HUC8,HUC8Description,Latitude,LifeStageCode,Longitude,ReportingUnits,ReportingValue,SampleType,State,Station,TSN,Time,Point,HUC12_,HUCNAME_,FIPS_,COUNTY_,STATE_


In [60]:
cbp_benthic.sort_values(by='EventId')[['EventId', 'Benthic Classification Name', 'ReportingValue']]

,EventId,Benthic Classification Name,ReportingValue
9351,74157.0,Sperchon,2.0
9376,74157.0,Crangonyx,1.0
9373,74157.0,Hydropsyche,2.0
9370,74157.0,Clinocera,3.0
9368,74157.0,Oligochaeta,16.0
...,...,...,...
123230,95779.0,Probezzia,3.0
123229,95779.0,Ceratopsyche,10.0
123228,95779.0,Epeorus,9.0
123237,95779.0,Leuctra,11.0


In [13]:
cbp_benthic.to_csv('/Users/bibor/Hack_the_Bay/Data/CBP_Benthic_clean.csv', index=False)

#### CMC Benthic Data

In [9]:
cmc_benthic = pd.read_excel(
    '/Users/bibor/Hack_the_Bay/Data/CMC_Benthic_Data.xlsx',
    sheet_name=0,
    engine='xlrd')

In [10]:
cmc_benthic.shape

(122582, 29)

In [11]:
cmc_benthic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122582 entries, 0 to 122581
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   EVENT_ID               122582 non-null  int64  
 1   SAMPLE_NUMBER          122582 non-null  int64  
 2   STATION_ID             122582 non-null  object 
 3   ICPRB_BIOREGION_ID     122582 non-null  object 
 4   STRAHLER_STREAM_ORDER  0 non-null       float64
 5   G_Method               122582 non-null  int64  
 6   SAMPLE_DATE            122582 non-null  object 
 7   MONTH                  122582 non-null  int64  
 8   Latitude               122582 non-null  object 
 9   Longitude              122582 non-null  float64
 10  FINAL_ID               122519 non-null  object 
 11  REPORTING_VALUE        122519 non-null  object 
 12  TSN_FINAL              122519 non-null  object 
 13  TAXON_LEVEL            122519 non-null  object 
 14  PHYLUM                 122519 non-nu

Note: TSN_FINAL should be a seven digit number, TAXON_LEVEL is FAM, 
source: https://www.chesapeakebay.net/documents/3683/benthic_rdbms_sql_7-29-10.pdf

Inspecting the cmc_benthic file in excel revealed that latitude and longitude columns must have been inserted by mistake - data up to index 41352 (included) has two extra columns (latitude and longitude) compared to data starting from index 41353. Hence, values are missing in the last two columns (BIBI_FFG and BIBI_HABIT) starting from index 41353.

In [12]:
# select first part of data (up to index 41352)
first_part = cmc_benthic.iloc[:41353]

In [13]:
# drop Latitude and Longitude from first part of dataframe
clean_first = first_part.drop(['Latitude', 'Longitude'],
                axis=1)

In [14]:
# select second part of dataset (from index 41353)
second_part = cmc_benthic.iloc[41353:]

In [15]:
# rename columns in second part: delete column names Latitude and Longitude, and shift all other column names left

# create list of columns
columns_old = list(second_part.columns)
columns_new = list(second_part.columns[:8].append(second_part.columns[10:])) + ['Extra1', 'Extra2']

# rename columns
clean_second = second_part.rename(columns={old: new for (old, new) in zip(columns_old, columns_new)})

In [16]:
# drop columns Extra1 and Extra2
clean_second.drop(columns=['Extra1', 'Extra2'], inplace=True)

In [17]:
# merge clean_first and clean_second
clean_benthic = clean_first.append(clean_second, verify_integrity=True)

In [18]:
# where 'FAMILY' is 'UNIDENTIFIED' = the organism is unidintified, and measured values are missing (ASPT, BIBI_TV, etc.) (448 rows)
# dropping those rows from dataframe
clean_benthic = clean_benthic.loc[clean_benthic['FAMILY'] != 'UNIDENTIFIED']

In [19]:
# clean_benthic.info()

In [20]:
# all values of column STRAHLER_STREAM_ORDER are missing
# dropping column STRAHLER_STREAM_ORDER
clean_benthic.drop(columns='STRAHLER_STREAM_ORDER', inplace=True)

**Benthic Metadata**

In [21]:
cmc_benthic_meta = pd.read_excel(
    '/Users/bibor/Hack_the_Bay/Data/CMC_Benthic_Data.xlsx',
    sheet_name=1,
    engine='xlrd')

In [22]:
cmc_benthic_meta.shape

(122582, 4)

In [23]:
# create dataframe with unique station_id and corresponding Lat/ Long data
unique_stations = cmc_benthic_meta.drop_duplicates('STATION_ID')

In [24]:
# drop ICPRB_BIOREGION_ID column
unique_stations.drop(columns='ICPRB_BIOREGION_ID', inplace=True)

/Users/bibor/opt/anaconda3/envs/hackbay/lib/python3.7/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


**Merge dataframes cmc_benthic and cmc_benthic_meta**

In [25]:
merged_cmc_benthic = pd.merge(clean_benthic,
                              unique_stations,
                              how='left',
                              on=['STATION_ID'],
                              validate='many_to_one'
                              )

In [26]:
# save merged_cmc_benthic as csv file

# merged_cmc_benthic.to_csv('/Users/bibor/Hack_the_Bay/Data/Merged_Benthic.csv', index=False)

**Check data - is ecoli present**

In [27]:
def check_ecoli(column):
    '''function to check 
        if ecoli is among the values of a column'''
    # mask: transform value into lower case characters and look for 'ecoli' in string
    mask = merged_cmc_benthic[column].str.lower().str.contains('ecoli', na=False)
    # return dataframe where mask is True
    return merged_cmc_benthic.loc[mask]

In [28]:
# check all columns where data type is string
for column in merged_cmc_benthic.columns[10:20]:
    if merged_cmc_benthic[column].dtype != 'int64':
        print(column, 'checked')
        check_ecoli(column)

TAXON_LEVEL checked
PHYLUM checked
SUBPHYLUM checked
CLASS checked
SUBCLASS checked
ORDER checked
SUBORDER checked
FAMILY checked
SUBFAMILY checked
TRIBE checked


No ecoli in cmc_benthic either. 